In [277]:
import numpy as np
from scipy.optimize import minimize
from scipy.io import loadmat
from numpy.linalg import det, inv
from math import sqrt, pi
import scipy.io
import matplotlib.pyplot as plt
import pickle
import sys

from scipy.stats import multivariate_normal

In [350]:
def ldaLearn(X,y):
    # Inputs
    # X - a N x d matrix with each row corresponding to a training example
    # y - a N x 1 column vector indicating the labels for each training example
    #
    # Outputs
    # means - A d x k matrix containing learnt means for each of the k classes
    # covmat - A single d x d learnt covariance matrix
    
    # N: training examples
    # d: features
    # k: classes
    
    N, d = X.shape
    unique_classes = np.unique(y) # grab each unique value
    k = unique_classes.size
    
    _means = []
    for uc in unique_classes:
        is_class = (y == uc).reshape(y.shape[0], ) # condition-matrix where y is the class
        from_class = X[is_class]                   # grab from X where y is the class
        class_means = np.mean(from_class, axis=0)  # get the means for each column (d columns)
        _means.append(class_means)

    means = np.array(_means).T # reshape means to be d x k (versus k x d)
    
    covmat = np.cov(X.T)
    
    return means,covmat

In [351]:
def ldaTest(means,covmat,Xtest,ytest):
    # Inputs
    # means, covmat - parameters of the LDA model
    # Xtest - a N x d matrix with each row corresponding to a test example
    # ytest - a N x 1 column vector indicating the labels for each test example
    # Outputs
    # acc - A scalar accuracy value
    # ypred - N x 1 column vector indicating the predicted labels

    res = np.zeros(len(ytest))
    for i, x in enumerate(Xtest):
        y = ytest[i]
        values = []
        for mean in means.T:
            values.append(multivariate_normal.pdf(x, mean, covmat))

        res[i] = np.argmax(np.array(values)) + 1 # we add one to match index
        
    acc = sum(res - ytest.reshape(len(ytest), ) == 0)/len(res)
            
    return acc, np.array(res).reshape(ytest.shape)

In [352]:
if sys.version_info.major == 2:
    X,y,Xtest,ytest = pickle.load(open('sample.pickle','rb'))
else:
    X,y,Xtest,ytest = pickle.load(open('sample.pickle','rb'),encoding = 'latin1')
    
means,covmat = ldaLearn(X,y)
ldaacc,ldares = ldaTest(means,covmat,Xtest,ytest)
print('LDA Accuracy = '+str(ldaacc))

LDA Accuracy = 0.97
